# Scrape

**Jupyter notebook that scrapes domain.com.au and outputs the information collected into parquet files**

Below are some functions that will help with scraping

In [18]:
def get_chunks(suburbs_df) -> dict:
    """function that splits up postcode dataframe into chunks of 25 so that if we are kicked halfway during scraping we don't lose too much progress

    parameters: 
    suburbs_df -> spark dataframe containing postcode data which we are wanting to split up into chunks to assist with scraping

    output:
    chunk_dict -> dictionary containing chunked up postcodes which will be used when we scrape our data
    """
    chunk_dict = {}
    
    i = 3048
    j = 3023  
    while i < 3997:
        temp = suburbs_df[suburbs_df['postcode'] > j]
        chunk_dict['chunk_{}'.format(i)] = temp[temp['postcode'] <= i]
        j += 25
        i += 25

    return chunk_dict



In [19]:
# Working METHOD
import re
from json import dump
from tqdm import tqdm
from collections import defaultdict
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
import pandas as pd

# Initialize Spark session
spark = SparkSession.builder.master('local[*]') \
    .config("spark.driver.memory", "15g") \
    .config("spark.executor.memory", "16g") \
    .appName("PropertyScraper") \
    .getOrCreate()

# Constants
BASE_URL = "https://www.domain.com.au"
N_PAGES = range(1, 50)  # Max number of pages you want to scrape  

# Load suburbs CSV
suburbs_df = pd.read_csv('postcodes.csv')  # Ensure this CSV contains 'suburb' and 'postcode' columns
chunk_dict = get_chunks(suburbs_df)

def start_scrape(chunk: dict, file_suffix: int) -> None:
    """Function that scrapes https://www.domain.com.au and outputs the data into a JSON file
    
    parameters:
    chunk: chunk of 50 postcodes we will scrape
    file_suffix: what we want to title the end of our files when we write to parquet (eg. chunk_{file_suffix}.parquet)

    output:
    No output, 
    """

    # Define schema for the Spark DataFrame
    schema = StructType([
        StructField("url", StringType(), True),
        StructField("postcode", StringType(), True),
        StructField("suburb", StringType(), True),
        StructField("name", StringType(), True),
        StructField("cost_text", StringType(), True),
        StructField("beds", StringType(), True),  # Separate field for beds
        StructField("baths", StringType(), True),  # Separate field for baths
        StructField("parking", StringType(), True),  # Parking field
        StructField("property_type", StringType(), True),  # Property type field
    ])


    # Initialize an empty DataFrame with the schema
    property_metadata = spark.createDataFrame([], schema)

    # Loop through each suburb and its postcode
    for index, row in chunk.iterrows():
        suburb = row['locality'].lower().replace(' ', '-')  # Convert to lowercase and hyphenate
        postcode = row['postcode']

        print(f"Scraping data for {suburb} ({postcode})")

        url_links = []
        page_found = False  # This flag will help us track whether any results are found

        # Generate list of URLs to visit
        for page in N_PAGES:
            url = BASE_URL + f"/rent/{suburb}-vic-{postcode}/?ssubs=0&sort=suburb-asc&page={page}"
            try:
                bs_object = BeautifulSoup(urlopen(Request(url, headers={'User-Agent': "PostmanRuntime/7.6.0"})), "lxml")

                # Check if the page has results or shows "No results found"
                no_results = bs_object.find(text=re.compile("No results found", re.I))
                if no_results:
                    print(f"No results found for {suburb} on page {page}. Stopping further scraping for this suburb.")
                    break  # Exit the pagination loop if no results are found

                # Find property links
                index_links = bs_object.find("ul", {"data-testid": "results"})
                if not index_links:
                    print(f"No more results for {suburb} on page {page}.")
                    break  # Exit pagination if no results list is found (end of pages)

                index_links = index_links.findAll("a", href=re.compile(f"{BASE_URL}/*"))
                page_found = True  # At least one result was found on this page

                for link in index_links:
                    # If it's a property address, add it to the list
                    if 'address' in link.get('class', []):
                        url_links.append(link['href'])

            except Exception as e:
                print(f"Error fetching {url}: {e}")
                break  # Stop if there's an issue with fetching the page

        if not page_found:
            print(f"No results for {suburb}. Moving to the next suburb.")
            continue  # Skip to the next suburb if no pages were found for this one

        # For each URL, scrape some basic metadata
        pbar = tqdm(url_links)
        success_count, total_count = 0, 0

        for property_url in pbar:
            try:
                bs_object = BeautifulSoup(urlopen(Request(property_url, headers={'User-Agent': "PostmanRuntime/7.6.0"})), "lxml")
                total_count += 1

                # Get property name
                name = bs_object.find("h1", {"class": "css-164r41r"}).text.strip()

                # Get cost text
                cost_text = bs_object.find("div", {"data-testid": "listing-details__summary-title"}).text.strip()

                # Get rooms (beds and baths)
                rooms = bs_object.find("div", {"data-testid": "property-features"}).findAll(
                    "span", {"data-testid": "property-features-text-container"}
                )

                # Initialize variables
                beds, baths, parking = None, None, '0'  # Default value for parking is '0 Car'

                for feature in rooms:
                    text = feature.text
                    if 'Bed' in text:
                        beds_match = re.findall(r'\d+', text)
                        if beds_match:
                            beds = beds_match[0]  # Extract the number of beds
                    elif 'Bath' in text:
                        baths_match = re.findall(r'\d+', text)
                        if baths_match:
                            baths = baths_match[0]  # Extract the number of baths
                    elif 'Car' in text or 'Parking' in text:
                        parking_match = re.findall(r'\d+', text)
                        if parking_match:
                            parking = parking_match[0]  # Extract the number of parking spaces

                property_type_container = bs_object.find("div", {"data-testid": "listing-summary-property-type"})
                property_type = property_type_container.get_text(strip=True)

                # Create a row and append it to the DataFrame
                row = [(property_url, postcode, suburb, name, cost_text, beds, baths, parking, property_type)]
                row_df = spark.createDataFrame(row, schema)
                property_metadata = property_metadata.union(row_df)
                success_count += 1

            except AttributeError:
                print(f"Error scraping {property_url}: missing data")

            pbar.set_description(f"{(success_count / total_count * 100):.0f}% successful")

        # Show the DataFrame to ensure data is being appended
        #property_metadata.show()

    # Output to parquet file
    try:
        property_metadata.write.mode("overwrite").parquet('../data/raw/work_{}.parquet'.format(file_suffix))
        print(f"Data successfully written")
    except Exception as e:
       print(f"An error occured: {e}")

    #added this print statement so that the cell output can be scrollable - it's getting annoying to click the scroll bar >:(
    print("chunk finished")
    #return property_metadata


In [16]:
def run_chunk(starting_chunk: int) -> None:
    """Function that scrapes domain.com.au in chunks of 25 postcodes 7 times (split amongst group members)
    
    Parameters:
    starting_chunk -> starting chunk number that we want

    Return:
    None 
    """
    i = starting_chunk
    # we are running chunks of 25 postcodes 7 times each
    while i < starting_chunk + 175:
        start_scrape(chunk_dict["chunk_{}".format(i)], i) #i.split("_")[1])
        i += 25
    if i == 3923:
        start_scrape(chunk_dict["chunk_3923"], 3923)
        temp = suburbs_df[suburbs_df['postcode'] >= i + 1]
        chunk_dict['chunk_3996'] = temp[temp['postcode'] < 3997]
        start_scrape(chunk_dict['chunk_3996'], 3996)

**Down below we split up the scraping process between team members to scrape more efficiently and minimise loss of time if hardware limitations occur**

In [ ]:
#Davyn
starting_chunk = 3148
run_chunk(starting_chunk)

In [ ]:
#Arpan
starting_chunk = 3048 + 175
run_chunk(starting_chunk)

In [ ]:
#Priscilla
starting_chunk = 3048 + 350
run_chunk(starting_chunk)

In [ ]:
#Rachel
starting_chunk = 3048 + 525
run_chunk(starting_chunk)

In [ ]:
#Nathan
starting_chunk = 3048 + 700
run_chunk(starting_chunk)

We have to scrape the chunk that includes malvern separately as we incur connection errors whenever we try access malvern

In [ ]:
#scrape for suburbs in this chunk except for malvern as the connection breaks 
temp = suburbs_df[suburbs_df['postcode'] >= 3123]
chunk_dict['chunk_3148'] = temp[temp['postcode'] <= 3148]
start_scrape(chunk_dict['chunk_3148'], 3148)